In [ ]:
import rdflib
import numpy as np
import torch
from pykeen.triples import TriplesFactory
from pykeen.models.inductive import InductiveNodePieceGNN
from pykeen.losses import NSSALoss
from torch.optim import Adam
from pykeen.training import SLCWATrainingLoop
from pykeen.evaluation import RankBasedEvaluator

import torch.nn.functional as F
from modular_methods.similarity_utils import compute_cosine_similarity, match_entities
from modular_methods.graphToText_utils import get_literals_for_entities
from modular_methods.dedup_pipeline import deduplicate_graphs, save_matches
from modular_methods.output_utils import build_final_result

### ---- 1. Load RDF graphs ----

train_graph = rdflib.Graph()
test_clean = rdflib.Graph()
test_graph = rdflib.Graph()

train_graph.parse("data/train_combined_NodePiece.ttl")
test_clean.parse("data/healthcare_graph_test_clean.ttl")
test_graph.parse("data/healthcare_graph_replaced_high.ttl")

# Combine for inductive training/testing

test_combined = test_clean + test_graph
test_combined.serialize(destination="data/test_combined_NodePiece.ttl", format="turtle")
### ---- 2. Convert graphs to triples arrays ----

def graph_to_triples(g):
    return [
        (str(s), str(p), str(o))
        for s, p, o in g
        if isinstance(s, rdflib.URIRef) and isinstance(o, rdflib.URIRef)
    ]

train_triples = np.array(graph_to_triples(train_graph))
test_triples = np.array(graph_to_triples(test_combined))


In [34]:
train_triples2 = [(str(s), str(p), str(o)) for s, p, o in train_graph]
test_triples2 = [(str(s), str(p), str(o)) for s, p, o in test_graph]

In [35]:
# All unique entities and relations
entities = set()
relations = set()

for (h, r, t) in train_triples2 + test_triples2:
    entities.add(h)
    entities.add(t)
    relations.add(r)


In [36]:
entities

{'http://example.org/Person/30c638c7-dbd7-4ab3-ab2b-6b2fbb3f7310',
 '67e81409-4416-40a8-b858-b1283289d984',
 '55a3cb4c-7aa6-477e-9ea8-820c0c48819b',
 'http://example.org/ContactPoint/edce3d76-e53c-4a29-8d28-deb89bc18a33',
 'brianclayton@healthcare.org',
 'http://example.org/Address/9f6622e9-d6b2-41e3-a27b-f978777f4a56',
 'Smith.Primary@dept.healthcare.org',
 '6173fade-5928-4966-99c2-f9c5b8e87ae6',
 'http://example.org/Person/f8d8aa35-0662-4d25-b1dc-69d9f266ca38',
 'http://example.org/Person/ded198aa-d04c-4c38-b94e-e8f9ee7a8d96',
 'Arthritis Specialist',
 '25338201-2d71-47ec-ac18-935a6ae30829',
 '2328 GS',
 'ddbba0d7-af0b-453f-b5c6-a05da6e4a329',
 'Tara Johnson',
 '727cc9e6-3904-4182-9a43-b266601e40bf',
 'Obstetirc',
 'GYN',
 'f771141a-172d-48d3-a6b9-506ffcce5ea5',
 '8db3b86e-2c55-4b06-b7f8-0b4da1328a13',
 'http://example.org/Person/4694572e-4aab-4296-a866-365afb4d9ca6',
 '(0303) 524308',
 '948551a6-5402-48ed-bb56-1d1973418c71',
 'http://example.org/Person/7793a051-aa5d-4cc4-9b7a-cf57df

In [2]:
train_triples

array([['http://example.org/Person/b94cff8a-e336-403c-8fbd-d3f73d675430',
        'https://schema.org/name', 'Calvin Barr'],
       ['http://example.org/Address/e41e0111-74cf-4b94-8c89-f3182e22365c',
        'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
        'https://schema.org/PostalAddress'],
       ['http://example.org/Person/7e7a5de8-0ac6-4aa7-9140-85c9eb4b3e84',
        'http://www.w3.org/2000/01/rdf-schema#label', 'Mary Mcgee'],
       ...,
       ['http://example.org/ContactPoint/aacf7c01-6bf7-411e-a545-aafca2a8d7f7',
        'https://schema.org/identifier',
        'aacf7c01-6bf7-411e-a545-aafca2a8d7f7'],
       ['http://example.org/Person/d6f92e93-f597-434a-a814-91088c46898b',
        'https://schema.org/email', 'ashleyyroy@healthcare.org'],
       ['http://example.org/Person/1bc7e1ca-dddc-41aa-b519-436750dcea11',
        'https://schema.org/worksFor',
        'http://example.org/HealthcareOrganization/0b5aafef-85a1-482a-8761-585b4168705a']],
      dtype='<U78')

In [3]:
test_triples

array([['http://example.org/Person/dde3dd5d-0114-4a11-b79b-b9441c7b1e91',
        'https://schema.org/name', 'Christopher Barnes'],
       ['http://example.org/Person/27429e56-f0fc-464a-b0b9-08df9b6f8426',
        'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
        'https://schema.org/Person'],
       ['http://example.org/Person/0b3fc9ec-5cfb-4488-9e4c-8b07c5accbbf',
        'https://schema.org/identifier',
        '0b3fc9ec-5cfb-4488-9e4c-8b07c5accbbf'],
       ...,
       ['http://example.org/Person/538cdac0-51e0-45eb-a5ff-5d17ddbf9191',
        'https://schema.org/identifier',
        '538cdac0-51e0-45eb-a5ff-5d17ddbf9191'],
       ['http://example.org/Person/2d5b4522-d0c3-40f1-86bf-2c63179414b6',
        'https://schema.org/name', 'Renee Johnson'],
       ['http://example.org/Person/76444588-9d87-4c09-bc03-dec5c4848714',
        'https://schema.org/jobTitle', 'Geneticist']], dtype='<U78')

In [4]:
# Extract unique relations from train and test triples
train_relations = set(train_triples[:, 1])
test_relations = set(test_triples[:, 1])

# Find common and unique relations
common_relations = train_relations & test_relations
train_only_relations = train_relations - test_relations
test_only_relations = test_relations - train_relations

print(f"Number of relations in train: {len(train_relations)}")
print(f"Number of relations in test: {len(test_relations)}")
print(f"Number of common relations: {len(common_relations)}")
print(f"Relations only in train: {train_only_relations}")
print(f"Relations only in test: {test_only_relations}")

Number of relations in train: 23
Number of relations in test: 22
Number of common relations: 22
Relations only in train: {'http://www.w3.org/2002/07/owl#sameAs'}
Relations only in test: set()


In [5]:
tf_train = TriplesFactory.from_labeled_triples(train_triples, create_inverse_triples=True)
tf_train


TriplesFactory(num_entities=14060, num_relations=46, create_inverse_triples=True, num_triples=33905)

In [7]:
### ---- 3. Build TriplesFactory objects ----

tf_train = TriplesFactory.from_labeled_triples(train_triples, create_inverse_triples=True)
tf_test = TriplesFactory.from_labeled_triples(
    test_triples,
    relation_to_id=tf_train.relation_to_id,
    create_inverse_triples=True
)


### ---- 4. Train InductiveNodePieceGNN ----

model = InductiveNodePieceGNN(
    triples_factory=tf_train,
    inference_factory=tf_test,
    num_tokens=12,
    aggregation="mlp",
    embedding_dim=128,
    interaction="DistMult",
    loss=NSSALoss(margin=15),
    random_seed=42,
).to("cuda" if torch.cuda.is_available() else "cpu")

optimizer = Adam(model.parameters(), lr=0.0005)

training_loop = SLCWATrainingLoop(
    triples_factory=tf_train,
    model=model,
    optimizer=optimizer,
    mode="training"
)

print("Training NodePiece...")
training_loop.train(tf_train, num_epochs=10)

No symbolic computation of output shape.                       
No symbolic computation of output shape.                       


Training NodePiece...


Training epochs on cpu: 100%|██████████| 10/10 [33:34<00:00, 201.45s/epoch, loss=0.122, prev_loss=0.125]


[2.0137204700572924,
 0.22296439478117408,
 0.1744611435993151,
 0.15879223888006175,
 0.14880452003104216,
 0.1368983072144064,
 0.13134312322081038,
 0.1306080201951166,
 0.12490982023265326,
 0.12245203046636148]

In [16]:
from pykeen.typing import TESTING, TRAINING

In [17]:
test_evaluator = RankBasedEvaluator(
        mode=TESTING,
        metrics=["hits_at_k"] * 5,
        metrics_kwargs=[dict(k=k) for k in (1, 3, 5, 10, 100)],
        add_defaults=True
    )




In [ ]:
tf_test.mapped_triples
# This displays the mapped triples tensor for the test set.
# Each row is a triple (head_id, relation_id, tail_id) where the entities and relations are mapped to their integer IDs
# according to the TriplesFactory's internal vocabulary.
# This is the input format used by PyKEEN models for evaluation and prediction.

tensor([[4047,    0, 5609],
        [4047,    4, 2723],
        [4047,    5, 1911],
        ...,
        [5604,   13, 4000],
        [5604,   17, 2825],
        [5604,   18, 4586]])

In [24]:
# For PyKEEN-like models
entity_embeddings = model.entity_representations[0]().detach().cpu().numpy()

# You can map from internal ID back to the original entity:
entity_id_to_label = tf_test.entity_id_to_label  # {id: 'entity_label'}
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# similarity_matrix[i, j] = similarity between entity i and entity j
similarity_matrix = cosine_similarity(entity_embeddings)

# Get entity labels for reference
entity_labels = [entity_id_to_label[i] for i in range(len(entity_id_to_label))]

entity_labels

['(000) 0035114',
 '(0028)-499243',
 '(003)-8016162',
 '(0053) 573452',
 '(007) 7131339',
 '(0089) 969726',
 '(0097)-410786',
 '(0110) 438805',
 '(0145) 901789',
 '(0150)-493331',
 '(0164) 630310',
 '(019) 0391103',
 '(022)-5402417',
 '(0263) 334006',
 '(029) 7657829',
 '(0329) 762146',
 '(033) 6184256',
 '(0338) 007752',
 '(035) 7800046',
 '(0359)-559906',
 '(045) 1726813',
 '(0508) 687889',
 '(0509) 558393',
 '(051) 3322487',
 '(059)-4724061',
 '(062)-7912560',
 '(063) 7805164',
 '(0670)-172009',
 '(0675) 727514',
 '(0714)-163115',
 '(076) 7970617',
 '(0806) 446410',
 '(0874)-206546',
 '(0875) 267737',
 '(0884) 832251',
 '(0904) 980790',
 '(0918)-558554',
 '(0921)-163416',
 '(0970)-341489',
 '(099) 9073342',
 '(0997)-391880',
 '(203)841-5253',
 '(218)919-2132x39644',
 '(274)327-9086',
 '(280)351-9921x4987',
 '(314)574-2692x96322',
 '(365)251-0888x770',
 '(392)794-5067x820',
 '(410)919-0400x6898',
 '(445)726-5176',
 '(463)307-4285',
 '(486)630-9419',
 '(591)818-7210x867',
 '(596)958-8

In [18]:

results = test_evaluator.evaluate(
    model=model,
    mapped_triples=tf_test.mapped_triples,
    batch_size=128,

)
print("Evaluation results:", results)

The filtered setting was enabled, but there were no `additional_filter_triples`
given. This means you probably forgot to pass (at least) the training triples. Try:

    additional_filter_triples=[dataset.training.mapped_triples]

Or if you want to use the Bordes et al. (2013) approach to filtering, do:

    additional_filter_triples=[
        dataset.training.mapped_triples,
        dataset.validation.mapped_triples,
    ]

Evaluating on cpu:   0%|          | 0.00/13.5k [00:00<?, ?triple/s]Encountered tensors on device_types={'cpu'} while only ['cuda'] are considered safe for automatic memory utilization maximization. This may lead to undocumented crashes (but can be safe, too).
Evaluating on cpu: 100%|██████████| 13.5k/13.5k [00:22<00:00, 612triple/s]

Evaluation results: <pykeen.evaluation.rank_based_evaluator.RankBasedMetricResults object at 0x000001A71A428C10>


In [26]:
import pandas as pd
golden_df = pd.read_csv("data/test_golden_standard_high.csv")
golden_pairs = set(zip(golden_df['original_id'], golden_df['duplicate_id']))
# Optionally, add the reverse direction if your duplicates are symmetric
golden_pairs |= set((b, a) for (a, b) in golden_pairs)

In [32]:
golden_pairs

{('28d733b5-b19d-43ff-b397-c29390f8eef3',
  '96e743ae-f3dc-4608-b3f5-5dae23bbbc63'),
 ('daac3c58-e266-44c0-9848-d32e65f628ac',
  '753f6426-15d7-4002-a974-d22825542d2a'),
 ('3b6b8130-f687-4f7a-b683-4a01bdb5ff1d',
  '5aafa5fe-3e5f-4c46-b1e6-7b2e2f5893df'),
 ('30860f35-5c27-421d-80e5-1d91eafe7d43',
  '30085671-b314-4ead-83f6-626bc7a1dfa5'),
 ('3609a69d-3b08-402b-adbd-3b474750fc5e',
  '7e6541ff-bfd2-4bf7-9da3-31443ec13354'),
 ('7008269f-d696-4917-9929-876474ac6013',
  '603fa8f0-1549-4dee-830b-4d6d9575d095'),
 ('45eba2bf-a4b4-4ce3-b974-8dc4186a5f7f',
  '8689c3e1-a530-4719-9f67-df3d15febd74'),
 ('0c652bed-d266-4a35-b68c-ff26a5f73e27',
  '18ad5597-8099-4a05-a0cb-fcb1b4d365eb'),
 ('6be7c42c-4cbd-4e7a-a800-cdd91018f9ce',
  '7766f556-ce51-4ffc-943d-da2a7bacbd85'),
 ('260680dc-0a6f-4199-a1f6-524b784bcf39',
  '4468f3b1-5ecb-479e-841c-76e28de031c4'),
 ('fdc8d008-61aa-4069-8e3a-131cb8a7b622',
  'f117f274-1840-4ff8-966d-a609aa80ddfc'),
 ('542d6f5f-5522-4c02-82c0-9dfd4c916005',
  'b8d0b72f-3da8-4e21-9

In [29]:
threshold = 0.7  # Adjust as appropriate
predicted_pairs = set()

for i in range(len(entity_labels)):
    for j in range(i + 1, len(entity_labels)):
        if similarity_matrix[i, j] > threshold:
            pred_pair = (entity_labels[i], entity_labels[j])
            predicted_pairs.add(pred_pair)
            predicted_pairs.add((entity_labels[j], entity_labels[i]))  # make symmetric


In [31]:
predicted_pairs

{('6368 NR', 'Shila Ortiz'),
 ('brentjonnes@healthcare.org', 'Herbert Galloway'),
 ('Christopher Vasquez',
  'http://example.org/Person/caabf1a2-e31e-4d64-9f64-d38fbeeab4a5'),
 ('Darryl Bell', 'Ashley Williams'),
 ('Mary Porter',
  'http://example.org/Person/14574af2-6364-48bd-8ddc-7208dddac3ef'),
 ('1960-10-12', 'Gastroentjerologic'),
 ('brittanymiller@healthcare.org', '174cb217-f384-4161-abb4-c667b0b25e03'),
 ('http://example.org/Person/c3f6c4d6-b661-48c5-ac57-7082492062de',
  '8f3e36e8-5e7f-4670-a090-0959af77c208'),
 ('http://example.org/Person/0d9a6fd6-a9f2-4214-ba34-ab5f24ae704b',
  '01b7a7c7-d2a9-483e-b3a3-6fc5de974c55'),
 ('Victoria Ward',
  'http://example.org/Person/109ced8e-b455-4b19-9364-f2342f06ff0a'),
 ('3bc4b4b2-a018-4f61-b8b2-2eb38ededebd', 'Olafstraat 113'),
 ('f97fa87f-b4a5-4860-bba9-6f62161a314b', 'karenhoward@healthcare.org'),
 ('9d4779da-d237-4bbc-ae49-d70be503ef26', '83054'),
 ('http://example.org/Address/1ae3237f-aa88-4965-9654-7b4d317588ff',
  'brainsantiago@heal

In [30]:
# True Positives: Predicted and in gold
tp = predicted_pairs & golden_pairs
# False Positives: Predicted but not in gold
fp = predicted_pairs - golden_pairs
# False Negatives: In gold but not predicted
fn = golden_pairs - predicted_pairs

precision = len(tp) / (len(tp) + len(fp)) if (len(tp) + len(fp)) else 0.0
recall = len(tp) / (len(tp) + len(fn)) if (len(tp) + len(fn)) else 0.0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0

print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 score: {f1:.3f}")


Precision: 0.000
Recall: 1.000
F1 score: 0.000


In [19]:
# Mean Reciprocal Rank (MRR)
print("MRR:", results.get_metric("mean_reciprocal_rank"))

# Mean Rank (MR)
print("MR:", results.get_metric("mean_rank"))

# Hits@1, Hits@3, Hits@10
print("Hits@1:", results.get_metric("hits_at_1"))
print("Hits@3:", results.get_metric("hits_at_3"))
print("Hits@10:", results.get_metric("hits_at_10"))


MRR: 0.1433243453502655
MR: 128.29974365234375
Hits@1: 0.09061644342485015
Hits@3: 0.14859764671057502
Hits@10: 0.25131354991489674


In [20]:
flat_results = results.to_flat_dict()
print(flat_results) 

{'head.optimistic.hits_at_1': 0.11884851624361725, 'tail.optimistic.hits_at_1': 0.06238437060608303, 'both.optimistic.hits_at_1': 0.09061644342485015, 'head.realistic.hits_at_1': 0.11884851624361725, 'tail.realistic.hits_at_1': 0.06238437060608303, 'both.realistic.hits_at_1': 0.09061644342485015, 'head.pessimistic.hits_at_1': 0.11884851624361725, 'tail.pessimistic.hits_at_1': 0.06238437060608303, 'both.pessimistic.hits_at_1': 0.09061644342485015, 'head.optimistic.hits_at_3': 0.12521275808480722, 'tail.optimistic.hits_at_3': 0.17198253533634278, 'both.optimistic.hits_at_3': 0.14859764671057502, 'head.realistic.hits_at_3': 0.12521275808480722, 'tail.realistic.hits_at_3': 0.17198253533634278, 'both.realistic.hits_at_3': 0.14859764671057502, 'head.pessimistic.hits_at_3': 0.12521275808480722, 'tail.pessimistic.hits_at_3': 0.17198253533634278, 'both.pessimistic.hits_at_3': 0.14859764671057502, 'head.optimistic.hits_at_5': 0.13283504773181382, 'tail.optimistic.hits_at_5': 0.25234958928439283,

In [11]:


### ---- 5. Extract entity embeddings ----

def extract_embeddings(model, triples_factory, mode="training"):
    model.mode = mode
    emb_array = model.entity_representations[0]().detach().cpu().numpy()
    entities = list(triples_factory.entity_to_id.keys())
    return {e: emb_array[i] for i, e in enumerate(entities)}

main_embeddings = extract_embeddings(model, tf_train, mode="training")
test_embeddings = extract_embeddings(model, tf_test, mode="testing")

# Make sure only common entities/types are compared (as your pipeline does)
entity_ids1 = list(main_embeddings.keys())
entity_ids2 = list(test_embeddings.keys())

emb1 = torch.tensor([main_embeddings[e] for e in entity_ids1])
emb2 = torch.tensor([test_embeddings[e] for e in entity_ids2])

emb1 = F.normalize(emb1, p=2, dim=1)
emb2 = F.normalize(emb2, p=2, dim=1)

sim_matrix = compute_cosine_similarity(emb1, emb2)
matches = match_entities(sim_matrix, entity_ids1, entity_ids2, threshold=0.1, top_k=5)

# Literal-based filtering (as in your pipeline)
literals1 = get_literals_for_entities(train_graph, entity_ids1)
literals2 = get_literals_for_entities(test_graph, entity_ids2)
from modular_methods.similarity_utils import Levenshtein_filter
filtered_matches = Levenshtein_filter(matches, literals1, literals2, filter=False)

### ---- 7. Format and save results ----

final_result = build_final_result(
    filtered_matches,
    train_graph,
    test_graph,
    graph1_name="MainGraph",
    graph2_name="TestGraph"
)
save_matches(final_result, "NodePiece_dedup_results.json")
print(f"Saved results to NodePiece_dedup_results.json")

TypeError: 'NoneType' object is not iterable